In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/MURI Aug17 Thin Plate/Rowdy'

In [4]:
# !pip install smt

In [5]:
#Material Properties This link - https://www.mathworks.com/help/pde/ug/nonlinear-heat-transfer-in-a-thin-plate.html#heatTransferThinPlateExample-1
k = 400
rho = 8960
cp = 386
t_z = 0.01
stef_bolt = 5.670373e-8
hc = 1
Ta = 300
emiss = 0.5


In [6]:
loss_thresh = 20000
label = "3D_HTTP_rowdy"

x = np.linspace(0,1,100).reshape(-1,1)
y = np.linspace(0,1,100).reshape(-1,1)
t = np.linspace(0,1,100).reshape(-1,1) #t is actually from 0 to 3000, let us scale it to 0 to 1

X,Y,T = np.meshgrid(x,y,t)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xyt = np.hstack((X,Y,T))

initial_pts = np.logical_and(T==0,Y!=0).reshape(-1,)

DBC_pts = (Y == 0).reshape(-1,)


NBC_pts_x0 = (X == 0).reshape(-1,)
NBC_pts_x1 = (X == 1).reshape(-1,)

NBC_pts_y0 = (Y == 0).reshape(-1,)
NBC_pts_y1 = (Y == 1).reshape(-1,)

xyt_initial = xyt[initial_pts,:]
xyt_DBC = xyt[DBC_pts,:]

xyt_NBC_x0 = xyt[NBC_pts_x0,:]
xyt_NBC_x1 = xyt[NBC_pts_x1,:]

#xyt_NBC_y0 = xyt[NBC_pts_y0,:]
xyt_NBC_y1 = xyt[NBC_pts_y1,:]

u_initial = 300*np.ones((np.shape(xyt_initial)[0],1))
u_DBC = 1000*np.ones((np.shape(xyt_DBC)[0],1))

xyt_I_DBC = np.vstack((xyt_initial,xyt_DBC))
#xyt_NBC = np.vstack((xyt_NBC_1,xyt_NBC_2,xyt_NBC_3,xyt_NBC_4))
xyt_NBC_x = np.vstack((xyt_NBC_x0,xyt_NBC_x1))
#xyt_NBC_y = np.vstack((xyt_NBC_y0,xyt_NBC_y1))
xyt_NBC_y = np.vstack((xyt_NBC_y1))

u_I_DBC = np.vstack((u_initial,u_DBC))



lb_xyt = xyt[0]
ub_xyt = xyt[-1]

In [7]:
fea_data = scipy.io.loadmat('./../3D_HTTP_FEA.mat')
xy = fea_data['xy']
t = fea_data['t']/3000
xyt = np.zeros((497*101,3))
u_true = np.ones((497*101,1))


for i in range(101):
    t_temp = t[0,i]*np.ones((497,1))
    xyt[497*i:497*(i+1)] = np.hstack((xy,t_temp))
    u_true[497*i:497*(i+1)] = fea_data['u'][:,i].reshape(-1,1)
    #print(i)
#print(xyt)

xyt_test_tensor = torch.from_numpy(xyt).float().to(device)
u_true_norm = np.linalg.norm(u_true,2)

In [8]:
def trainingdata(N_D,N_N,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    #choose random N_u points for training
    idx = np.random.choice(xyt_I_DBC.shape[0], N_D, replace=False) 
    xyt_D = xyt_I_DBC[idx,:] #choose indices from  set 'idx' (x,t)
    u_D = u_I_DBC[idx].reshape(-1,1)      #choose corresponding u

    idx = np.random.choice(xyt_NBC_x.shape[0], N_D, replace=False) 
    xyt_Nx = xyt_NBC_x[idx,:] #choose indices from  set 'idx' (x,t)

    idx = np.random.choice(xyt_NBC_y.shape[0], N_D, replace=False) 
    xyt_Ny = xyt_NBC_y[idx,:] #choose indices from  set 'idx' (x,t)

    '''Collocation Points'''
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xyt_coll = lb_xyt + (ub_xyt - lb_xyt)*samples
    xyt_coll = np.vstack((xyt_coll, xyt_D,xyt_Nx,xyt_Ny)) # append training points to collocation points 

    return xyt_coll, xyt_D, u_D, xyt_Nx,xyt_Ny

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val,rowdy_terms):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()
     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   

        
        self.omega1 = Parameter(torch.ones((len(layers)-2,1))) 
        self.omega1.requiresGrad = True
        
        
        self.alpha = Parameter(torch.zeros(rowdy_terms,len(layers)-2))
        self.alpha.requiresGrad = True
        
        
        self.omega = Parameter((1/n_val)*torch.ones(rowdy_terms,len(layers)-2))
        self.omega.requiresGrad = True
        
        self.n = torch.tensor(n_val)
            
    'foward pass'
    def forward(self,xyt):
        if torch.is_tensor(xyt) != True:         
            xyt = torch.from_numpy(xyt)                
        
        ubxyt = torch.from_numpy(ub_xyt).float().to(device)
        lbxyt = torch.from_numpy(lb_xyt).float().to(device)
    
                      
        #preprocessing input 
        xyt = (xyt - lbxyt)/(ubxyt - lbxyt)
        
        #convert to float
        a = xyt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.omega1[i,0]*z)
            for j in range(rowdy_terms):
                a = a + self.alpha[j,i]*self.n*torch.sin((j+1)*self.n*self.omega[j,i]*z)
                
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_D(self,xyt_D,u_D):
                
        loss_bc = self.loss_function(self.forward(xyt_D), u_D)
                
        return loss_bc
    
    def loss_N(self,xyt_Nx,xyt_Ny,N_hat):
        
        g1 = xyt_Nx.clone()             
        g1.requires_grad = True
        u1 = self.forward(g1)
        
        u1_x_y_t = autograd.grad(u1,g1,torch.ones([xyt_Nx.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du1_dx = u1_x_y_t[:,[0]]
        
        g2 = xyt_Ny.clone()             
        g2.requires_grad = True
        u2 = self.forward(g2)
        
        u2_x_y_t = autograd.grad(u2,g2,torch.ones([xyt_Ny.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du2_dy = u2_x_y_t[:,[1]]
               
        loss_N1 = self.loss_function(du1_dx,N_hat)
        loss_N2 = self.loss_function(du2_dy,N_hat)
        
        #return loss_N1+loss_N2       
        return loss_N1 + loss_N2
    
    def loss_PDE(self, xyt_coll, f_hat):
        
        g = xyt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_y_t = autograd.grad(u,g,torch.ones([xyt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy_tt = autograd.grad(u_x_y_t,g,torch.ones(xyt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dt = u_x_y_t[:,[2]]
        
        d2u_dx2 = u_xx_yy_tt[:,[0]]
        d2u_dy2 = u_xx_yy_tt[:,[1]]    
        

        f = rho*cp*t_z*du_dt/3000 - k*t_z*(d2u_dx2+d2u_dy2) + 2*hc*(u-Ta) + 2*emiss*stef_bolt*(torch.pow(u,4)-Ta**4) 
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat):

        loss_D = self.loss_D(xyt_D,u_D)
        loss_N = self.loss_N(xyt_Nx,xyt_Ny,N_hat)
        loss_f = self.loss_PDE(xyt_coll,f_hat)
        
        loss_val = loss_D + loss_N + loss_f
        
        #print(self.iter,"loss_D:",loss_D.cpu().detach().numpy(),"loss_N:",loss_N.cpu().detach().numpy(),"loss_f:",loss_f.cpu().detach().numpy())
        
        return loss_val
          
    'test neural network'
    def test(self):
        u_pred = self.forward(xyt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
               
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [10]:
def train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    omega_val.append(PINN.omega.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xyt_coll_np_array, xyt_D_np_array, u_D_np_array,xyt_Nx_np_array,xyt_Ny_np_array = trainingdata(N_D,N_N,N_f,(reps)*22)

    xyt_coll = torch.from_numpy(xyt_coll_np_array).float().to(device)
    xyt_D = torch.from_numpy(xyt_D_np_array).float().to(device)
    u_D = torch.from_numpy(u_D_np_array).float().to(device)
    xyt_Nx = torch.from_numpy(xyt_Nx_np_array).float().to(device)
    xyt_Ny = torch.from_numpy(xyt_Ny_np_array).float().to(device)

    N_hat = torch.zeros(xyt_Nx.shape[0],1).to(device)    
    f_hat = torch.zeros(xyt_coll.shape[0],1).to(device)
    
    for i in range(max_iter):
        train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,i)

        loss_np = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])

    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [13]:
max_reps = 10 #10
max_iter = 100 #1000

train_loss_full = []
test_mse_full = []
test_re_full = []
alpha_full = []
omega_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

n_val = 1.0
rowdy_terms = 6

for reps in range(max_reps):
    print(label)
    print(reps)

    train_loss = []
    test_mse_loss = []
    test_re_loss = []

    alpha_val = []    
    omega_val = []

    torch.manual_seed(reps*36)
    N_D = 5000 #Total number of data points for 'y'
    N_N = 3500
    N_f = 10000#Total number of collocation points 

    layers = np.array([3,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers


    PINN = Sequentialmodel(layers,n_val,rowdy_terms)

    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())

    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.1, 
                              max_iter = 20, 
                              max_eval = 30, 
                              tolerance_grad = 1e-8, 
                              tolerance_change = 1e-8, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')


    nan_flag = train_model(max_iter,reps)

    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time
    alpha_full.append(alpha_val)
    omega_full.append(omega_val)



    #print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time,"alpha": alpha_full,"omega": omega_full,  "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

3D_HTTP_rowdy
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 1034684.94 Test MSE 316745.0362455329 Test RE 0.9976869028849932
1 Train Loss 570587.44 Test MSE 312602.04782212566 Test RE 0.9911406116192567
2 Train Loss 514153.72 Test MSE 263075.74414584204 Test RE 0.9092425807305725
3 Train Loss 381633.66 Test MSE 163027.76034999685

96 Train Loss 24150.934 Test MSE 3180.6100491867987 Test RE 0.09997572807844245
97 Train Loss 24032.0 Test MSE 3368.4268671806562 Test RE 0.10288520412455204
98 Train Loss 23828.008 Test MSE 3352.6095024334195 Test RE 0.10264335714276861
99 Train Loss 23631.104 Test MSE 3270.6750869692282 Test RE 0.10138134851454743
Training time: 651.73
3D_HTTP_rowdy
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Li

91 Train Loss 56043.46 Test MSE 8508.14494608839 Test RE 0.16351470011420546
92 Train Loss 55909.96 Test MSE 8526.643304927824 Test RE 0.16369235995086084
93 Train Loss 55764.613 Test MSE 8454.97081813036 Test RE 0.1630029328984526
94 Train Loss 54923.996 Test MSE 8681.8974444089 Test RE 0.1651759023039308
95 Train Loss 54089.453 Test MSE 8991.33930145159 Test RE 0.168093745974611
96 Train Loss 53615.18 Test MSE 8992.870679510046 Test RE 0.16810805997757042
97 Train Loss 53031.81 Test MSE 8774.079140976348 Test RE 0.1660504802059302
98 Train Loss 52719.027 Test MSE 8842.671212641279 Test RE 0.16669827314107993
99 Train Loss 52298.41 Test MSE 8777.527423659094 Test RE 0.16608310657840786
Training time: 645.73
3D_HTTP_rowdy
2
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias

87 Train Loss 47339.125 Test MSE 7824.027602702299 Test RE 0.15680306560396254
88 Train Loss 46799.44 Test MSE 7715.803257796131 Test RE 0.15571481524439088
89 Train Loss 46136.555 Test MSE 7542.7283716253105 Test RE 0.15395847346331168
90 Train Loss 45896.39 Test MSE 7435.568881630265 Test RE 0.15286091779129307
91 Train Loss 45253.035 Test MSE 7331.017524874488 Test RE 0.1517824263491251
92 Train Loss 44897.773 Test MSE 7449.30801744939 Test RE 0.15300207764635665
93 Train Loss 44720.926 Test MSE 7481.937750927038 Test RE 0.15333680417074552
94 Train Loss 44651.55 Test MSE 7487.288455680779 Test RE 0.15339162374816198
95 Train Loss 44651.55 Test MSE 7487.288455680779 Test RE 0.15339162374816198
96 Train Loss 44651.55 Test MSE 7487.288455680779 Test RE 0.15339162374816198
97 Train Loss 44651.55 Test MSE 7487.288455680779 Test RE 0.15339162374816198
98 Train Loss 44651.55 Test MSE 7487.288455680779 Test RE 0.15339162374816198
99 Train Loss 44651.55 Test MSE 7487.288455680779 Test RE 0.

83 Train Loss 43963.44 Test MSE 6614.063701320968 Test RE 0.14416955199340065
84 Train Loss 43631.81 Test MSE 6607.777654807745 Test RE 0.14410102589652057
85 Train Loss 42969.65 Test MSE 6332.766528792391 Test RE 0.14107046593589362
86 Train Loss 42365.83 Test MSE 6053.41564408632 Test RE 0.1379239256063398
87 Train Loss 41942.574 Test MSE 5876.280221891599 Test RE 0.13589097388624094
88 Train Loss 41566.984 Test MSE 5908.678069150853 Test RE 0.13626506457906506
89 Train Loss 41219.14 Test MSE 6002.70912528144 Test RE 0.13734504999406963
90 Train Loss 40897.11 Test MSE 6013.421408220619 Test RE 0.13746754661920532
91 Train Loss 40497.59 Test MSE 5995.662558379627 Test RE 0.13726441179739346
92 Train Loss 40022.04 Test MSE 5846.675374418587 Test RE 0.1355482305826086
93 Train Loss 39868.668 Test MSE 5704.757954217715 Test RE 0.13389303115009826
94 Train Loss 39775.58 Test MSE 5668.275719554486 Test RE 0.13346421790333757
95 Train Loss 39442.688 Test MSE 5559.5025357106615 Test RE 0.132

...

75 Train Loss 48365.297 Test MSE 7255.264591305129 Test RE 0.15099619025045416
76 Train Loss 47060.133 Test MSE 7357.586477643811 Test RE 0.15205722128025068
77 Train Loss 46520.1 Test MSE 7203.166027998778 Test RE 0.15045307719978804
78 Train Loss 45689.035 Test MSE 7125.037316366259 Test RE 0.14963491243337076
79 Train Loss 44931.477 Test MSE 6702.278353456791 Test RE 0.145127793550595
80 Train Loss 44485.496 Test MSE 6635.872861773779 Test RE 0.14440704813566266
81 Train Loss 44116.3 Test MSE 6697.005040762928 Test RE 0.14507068947426094
82 Train Loss 43717.92 Test MSE 6647.536602722132 Test RE 0.14453390310701883
83 Train Loss 43570.805 Test MSE 6593.608376684635 Test RE 0.14394644266335066
84 Train Loss 43334.574 Test MSE 6424.788736403619 Test RE 0.14209172543135595
85 Train Loss 42999.65 Test MSE 6348.965862726414 Test RE 0.14125078112794556
86 Train Loss 42350.81 Test MSE 6201.067494093264 Test RE 0.1395958769681296
87 Train Loss 41781.797 Test MSE 6080.945746567163 Test RE 0.1

70 Train Loss 85378.75 Test MSE 14953.454720920074 Test RE 0.21677546785387933
71 Train Loss 84726.34 Test MSE 14930.589918286074 Test RE 0.2166096725729821
72 Train Loss 83883.07 Test MSE 14882.415406426913 Test RE 0.21625993769119953
73 Train Loss 82620.945 Test MSE 14395.099767520556 Test RE 0.21268981906006187
74 Train Loss 81619.86 Test MSE 14017.461546859666 Test RE 0.2098814466043282
75 Train Loss 80937.09 Test MSE 14289.051526320245 Test RE 0.21190493126783883
76 Train Loss 80319.75 Test MSE 13819.534389298691 Test RE 0.20839441118269217
77 Train Loss 79045.2 Test MSE 12901.918458623566 Test RE 0.20135689638588028
78 Train Loss 75405.945 Test MSE 11390.084022733756 Test RE 0.18919202519369166
79 Train Loss 74510.836 Test MSE 11773.810907001478 Test RE 0.19235252448777826
80 Train Loss 74099.36 Test MSE 11294.345406810071 Test RE 0.1883952266237771
81 Train Loss 73934.72 Test MSE 10909.608715486402 Test RE 0.18515862602346886
82 Train Loss 73468.16 Test MSE 10838.463917087613 Te

66 Train Loss 136838.28 Test MSE 32128.82413426155 Test RE 0.31775089161619535
67 Train Loss 135244.19 Test MSE 31279.955104749897 Test RE 0.31352517774496086
68 Train Loss 132679.33 Test MSE 30834.407053136885 Test RE 0.31128426115649854
69 Train Loss 131734.17 Test MSE 30228.068467825342 Test RE 0.3082084637956754
70 Train Loss 130661.71 Test MSE 29452.750587267717 Test RE 0.30423017836460636
71 Train Loss 127810.86 Test MSE 28662.281570304764 Test RE 0.3001198641828894
72 Train Loss 125385.01 Test MSE 27972.03463524679 Test RE 0.296484088463784
73 Train Loss 123437.26 Test MSE 27328.65928688793 Test RE 0.2930545882160695
74 Train Loss 121370.43 Test MSE 26563.059283666753 Test RE 0.2889205333967201
75 Train Loss 120273.664 Test MSE 26169.560017308468 Test RE 0.28677254642744926
76 Train Loss 119383.98 Test MSE 25959.846520156938 Test RE 0.28562118881932147
77 Train Loss 117827.35 Test MSE 25297.971504907055 Test RE 0.2819565654518854
78 Train Loss 116395.14 Test MSE 25098.6326177524

61 Train Loss 39501.78 Test MSE 4952.379978275144 Test RE 0.12475166244971955
62 Train Loss 38746.496 Test MSE 5034.497639071669 Test RE 0.12578169213321197
63 Train Loss 38261.8 Test MSE 4928.539767283705 Test RE 0.12445102983843158
64 Train Loss 38112.027 Test MSE 4851.778786692494 Test RE 0.12347807715026264
65 Train Loss 37787.473 Test MSE 4826.897101116714 Test RE 0.12316104993386981
66 Train Loss 37096.25 Test MSE 4801.802984027616 Test RE 0.12284048735488146
67 Train Loss 36768.387 Test MSE 4793.336967827713 Test RE 0.12273215009549981
68 Train Loss 36650.645 Test MSE 4647.707636541438 Test RE 0.12085336929776079
69 Train Loss 36210.953 Test MSE 4497.452674083452 Test RE 0.11888379583924341
70 Train Loss 35711.957 Test MSE 4310.88115040907 Test RE 0.11639180065017232
71 Train Loss 35596.29 Test MSE 4313.440086645274 Test RE 0.11642634057364923
72 Train Loss 35399.953 Test MSE 4394.693915915805 Test RE 0.11751780698786204
73 Train Loss 35192.01 Test MSE 4456.234898628251 Test RE 

57 Train Loss 147100.7 Test MSE 34713.20007330722 Test RE 0.3302833553236383
58 Train Loss 146948.1 Test MSE 34695.19283769401 Test RE 0.33019767809291667
59 Train Loss 146908.88 Test MSE 34725.970855442356 Test RE 0.3303441043887479
60 Train Loss 146839.8 Test MSE 34823.04247946424 Test RE 0.33080549768373024
61 Train Loss 146538.42 Test MSE 34690.15767210069 Test RE 0.3301737171320797
62 Train Loss 146112.05 Test MSE 34312.13576712961 Test RE 0.32836982237143597
63 Train Loss 145715.14 Test MSE 34082.10754583103 Test RE 0.32726727736869465
64 Train Loss 145443.6 Test MSE 33706.37821674131 Test RE 0.32545834153158887
65 Train Loss 145020.19 Test MSE 33442.284792130646 Test RE 0.3241808321225992
66 Train Loss 144849.05 Test MSE 33423.15863257101 Test RE 0.3240881168501491
67 Train Loss 144579.28 Test MSE 33340.81728777169 Test RE 0.3236886586814668
68 Train Loss 144358.67 Test MSE 33069.16323738408 Test RE 0.3223672876512487
69 Train Loss 143784.55 Test MSE 33027.06427903753 Test RE 0.

In [14]:
import scipy.io as sio
import numpy as np

In [15]:
for tune_reps in range(50):
    label = "3D_HTTP_rowdy_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

FileNotFoundError: [Errno 2] No such file or directory: '3D_HTTP_rowdy_tune0.mat'

In [ ]:
lrnr_tune[30]